# Face recognition with ORL

In [1]:
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image
import os
import dlib
import cv2
import random

In [2]:
dsFolderPath = "ds_faces" 
rows = 0 
numberPhotoOnePerson = 0
digitalSurname = ""
photoname = ""
mmds = ""


In [3]:
def pathesOfImagesFromDs(dsFolderPath):
    global rows
    global numberPhotoOnePerson
    listOfSubfolders = os.listdir(dsFolderPath)
    photos = list()
    for i in listOfSubfolders:
        listOfImagesInCurrentSubfolder = os.listdir(dsFolderPath + "/" + i)
        for j in listOfImagesInCurrentSubfolder:            
            photos.append(dsFolderPath + "/" + i + "/" + j)
    rows = len(photos)
    photos.sort()
    numberPhotoOnePerson = len (listOfSubfolders)
    return rows, numberPhotoOnePerson, photos

In [4]:
#pathesOfImagesFromDs(dsFolderPath)

In [5]:
def initDigitalSurname():
    global digitalSurname
    global number_ds
    number_ds = numberPhotoOnePerson
    numberDigitalSurname = rows / number_ds # под одной цифровой фамилией будет
    digitalSurname = [0] * rows
    for i in range (0, number_ds, 1):
        for j in range (0, int(numberDigitalSurname), 1):
            digitalSurname [int(str(i) + str(j))] = i + 1
    return number_ds, numberDigitalSurname, len(digitalSurname), digitalSurname

In [6]:
#initDigitalSurname()

In [7]:
def getSizePhoto ():
    img = np.array(pathesOfImagesFromDs(dsFolderPath)[2])
    i = 0 # самое первое фото
    img2 = cv2.imread(img[i]) 
    h = img2.shape[0]
    w = img2.shape[1]
    c = img2.shape[2]
    n = h * w
    m = rows    
    return m, n

In [8]:
#getSizePhoto ()

In [9]:
def create_MMDS():
    m = getSizePhoto ()[0]
    n = getSizePhoto ()[1]
    mmds = np.zeros((m,n))
    return mmds       

In [10]:
def parsePhoto (i):
    im=np.array(Image.open(pathesOfImagesFromDs(dsFolderPath)[2][i]))
    im=im.reshape(1,im.shape[0]*im.shape[1])    
    return im

In [11]:
def create_MMDS():
    m = getSizePhoto ()[0]
    n = getSizePhoto ()[1]
    global mmds
    mmds = np.zeros((m,n)) 
    for i in range (0, m):
        mmds [i] = (parsePhoto (i))/255
    return mmds

In [12]:
#create_MMDS()

In [13]:
def numberSplitMMDS (numberTrain):
    numberTest = 10 - numberTrain
    return numberTrain, numberTest

In [14]:
#numberSplitMMDS (8)

In [15]:
def splitMMDS ():
    n = getSizePhoto ()[1] 
    m = getSizePhoto ()[0]
    i_trainMMDS = 0
    i_testMMDS = 0

    tM = 8
    mmds = create_MMDS()
    testM = int(initDigitalSurname()[1]) - tM
    trainMMDS = np.zeros(((tM * number_ds), n))
    vyb = int(int(initDigitalSurname()[1]) / testM)
    trainDigitalSurname = list()
    testMMDS = np.zeros(((testM * number_ds), n))
    testDigitalSurname = list()
    
    for i in range(0, m):
        if (i%vyb != 0):
            trainMMDS[i_trainMMDS,:] = mmds[i,:]
            i_trainMMDS += 1       
            trainDigitalSurname.append(digitalSurname[i])
        else:
            testMMDS[i_testMMDS,:] = mmds[i,:]    
            i_testMMDS += 1    
            testDigitalSurname.append(digitalSurname[i])
           
    return trainMMDS, trainDigitalSurname, testMMDS, testDigitalSurname

In [16]:
#splitMMDS ()

In [17]:
def evklidDistanse (vector1, vector2):
    return np.linalg.norm(vector2 - vector1)

In [18]:
def minDist():
    mmds1 = splitMMDS()[0] 
    trainDS = splitMMDS()[1] 
    mmds2 = splitMMDS()[2] 
    testDS = splitMMDS()[3] 
    acc = 0 
    vector1 = np.zeros (len(mmds1)) 
    vector2 = np.zeros (len(mmds2)) 
    iTest = 0
    for i in range (0, len(mmds2), 1): 
        iTest += 1
        minDist = 100
        for j in range (0, len(mmds1),): 
            vector1 = mmds1[j] 
            vector2 = mmds2[i] 
            evk = evklidDistanse (vector1, vector2)
            if (evk < minDist):
                minDist = evk
                jv = j
                evk = 0            
        if (testDS[i] == trainDS[jv]):
            acc += 1
    return (acc / iTest)

In [19]:
print ("accurancy = ", minDist())

accurancy =  0.975
